In [ ]:
!pip install ruamel.yaml

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

import torch
import glob
import os
#torch.load("")

def go(layers):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/dense_large.logit_lens.{}/*/*.pt".format(layers)):
        ch = torch.load(f, map_location="cpu")
        if 'args' in ch:
            del ch['optimizer']
            ch['args']['num_layers'] = layers
            os.rename(f, f + ".args_orig")
            torch.save(ch, f)
        del ch

In [ ]:
for i in range(25):
    go(i)

In [ ]:
for layers in range(25):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/dense_medium.{}/*/*.pt".format(layers)):
        ch = torch.load(f)
ch

In [ ]:
import ruamel.yaml
import sys

In [ ]:
import sys
from ruamel.yaml import YAML

inp = """\
{
   # parallelism settings ( you will want to change these based on your cluster setup, ideally scheduling pipeline stages
   # across the node boundaries )
   "pipe-parallel-size": 1,
}
"""

yaml = YAML()
code = yaml.load(inp)
yaml.dump(code, sys.stdout)

In [ ]:
import yaml
file_name = '/mnt/ssd-1/igor/gpt-neox/models/dense_medium/configs/config.yml'
with open(file_name) as conf_file:
    conf = yaml.load(conf_file, Loader=yaml.FullLoader)

print(conf['num-layers'])
conf['num-layers'] = 17
conf['load'] = "mypath"
conf['save'] = "mypath"

with open('/tmp/test.yml', 'w') as f:
    yaml.dump(conf, f)

In [ ]:
# Rename wandb runs

import wandb
import json
import os

api = wandb.Api()
for run in api.runs("igoro/neox"):
    if run.name != "distill-0-0":
        print("Skipping", run.name)
        continue

    j = json.loads(run.json_config)
    if "load" in j and "value" in j["load"]:
        new_name = os.path.basename(j["load"]["value"])
    else:
        print("WARNING: Skipping", run)
        continue
    
    print("Renaming", run.name)
    run.name = new_name
    run.update()


In [ ]:
import torch
import glob
import os

from collections import OrderedDict

def show(x, prefix = "", path = ""):
    if type(x) is dict or type(x) is OrderedDict:
        print(prefix, path)
        for key in x.keys():
            show(x[key], prefix + "  ", str(key))
    elif type(x) is torch.Tensor:
        print(prefix, path, x.shape, x.dtype)
    elif type(x) is list:
        print(prefix, path)
        for i in range(len(x)):
            show(x[i], prefix + "  ", "[{}]".format(i))
    else:
        print(prefix, path, type(x), x)

def go(model, checkpoint):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/{}/{}/*.pt".format(model, checkpoint)):
        ch = torch.load(f, map_location="cpu")
        print("----", f)
        if 'optimizer' in ch:
            #print(f, ch['optimizer']["optimizer_state_dict"].keys())
            show(ch)


def load(model, checkpoint):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/{}/{}/mp_*.pt".format(model, checkpoint)):
        ch = torch.load(f, map_location="cpu")
        return ch
#go("dense_small.final_linear.7", "global_step10000")
#go("dense_small.logit_lens.7", "global_step485000")


In [ ]:
#print(c1)
#print(c2)
#c2["optimizer"]["scale_window"] = c1["optimizer"]["scale_window"]


c1 = load("dense_small.final_linear.7", "global_step10000")
c2 = load("dense_small.logit_lens.7", "global_step485000")
#c2["optimizer"]["fp32_groups_flat"] = []
torch.save(c2, "/mnt/ssd-1/igor/gpt-neox/models/dense_small.final_linear.7_exp/baseline/mp_rank_00_model_states.pt")

#shutil.copy("/mnt/ssd-1/igor/gpt-neox/models/dense_small.final_linear.7/baseline/mp_rank_00_model_states.pt", "/mnt/ssd-1/igor/gpt-neox/models/dense_small.final_linear.7_exp/baseline/")
#shutil.copy("/mnt/ssd-1/igor/gpt-neox/models/dense_small.logit_lens.7/global_step485000/mp_rank_00_model_states.pt", "/mnt/ssd-1/igor/gpt-neox/models/dense_small.final_linear.7_exp/baseline/")

In [ ]:
c1 = load("dense_small.final_linear.7", "global_step10000")

In [ ]:
print(c1.keys())

In [ ]:
cc = torch.load("/mnt/ssd-1/igor/gpt-neox/models/dense_small/global_step485000/mp_rank_00_model_states.pt")

In [ ]:
show(cc)

In [ ]:
model = "dense_small.out_linear_all.0"
checkpoint = "baseline"
print("/mnt/ssd-1/igor/gpt-neox/models/{}/{}/*.pt".format(model, checkpoint))
for f in sorted(glob.glob("/mnt/ssd-1/igor/gpt-neox/models/{}/{}/*.pt".format(model, checkpoint))):
    ch = torch.load(f, map_location="cpu")
    print("----", f)
    show(ch)